In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customersloandata-csv/Task 3 and 4_Loan_Data.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the dataset
data = pd.read_csv('/kaggle/input/customersloandata-csv/Task 3 and 4_Loan_Data.csv')
data.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


## Using Random Forest

In [3]:
# Define features and target variable
X = data.drop(columns=['customer_id', 'default'])
y = data['default']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Function to predict the probability of default for a new loan
def predict_default_probability(loan_details):
    # loan_details is a dictionary containing loan details
    # Convert loan details to DataFrame with a single row
    loan_df = pd.DataFrame([loan_details])
    # Predict probability of default for the loan
    probability = clf.predict_proba(loan_df)[:, 1]
    return probability[0]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1652
           1       0.99      0.98      0.98       348

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [4]:
# Example usage of the predict_default_probability function
loan_details = {
    'credit_lines_outstanding': 0,
    'loan_amt_outstanding': 5221.545193,
    'total_debt_outstanding': 3915.471226,
    'income': 78039.38546,
    'years_employed': 5,
    'fico_score': 605
}

probability_of_default = predict_default_probability(loan_details)
print(f"Probability of default: {probability_of_default}")

Probability of default: 0.0


## Using Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np
import pandas as pd


# Define the variable features
features = ['credit_lines_outstanding', 'debt_to_income', 'payment_to_income', 'years_employed', 'fico_score']

# Calculate the payment_to_income ratio
data['payment_to_income'] = data['loan_amt_outstanding'] / data['income']
    
# Calculate the debt_to_income ratio
data['debt_to_income'] = data['total_debt_outstanding'] / data['income']

clf = LogisticRegression(random_state=0, solver='liblinear', tol=1e-5, max_iter=10000).fit(data[features], data['default'])
print(clf.coef_, clf.intercept_)

# Use the following code to check yourself
y_pred = clf.predict(data[features])

fpr, tpr, thresholds = metrics.roc_curve(data['default'], y_pred)
print((1.0*(abs(data['default']-y_pred)).sum()) / len(data))
print(metrics.auc(fpr, tpr))


[[ 8.18520373  0.54490854  0.01994244 -2.77630853 -0.02418391]] [-0.09162643]
0.0037
0.9925106069101026
